In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.createOrReplaceTempView('green')

In [4]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [5]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-22 16:00:00|  74|1107.7699999999988|            70|
|2020-01-13 20:00:00|   7|330.63000000000005|            27|
|2020-01-23 21:00:00|  25|196.79000000000005|            17|
|2020-01-10 18:00:00|  41| 728.5999999999997|            55|
|2020-01-09 10:00:00| 116|352.74000000000007|            20|
|2020-01-23 09:00:00|  75| 650.6299999999998|            55|
|2020-01-18 10:00:00|  42| 266.4900000000001|            24|
|2020-01-28 19:00:00|  75|1298.9999999999995|            97|
|2020-01-15 04:00:00| 173|              21.1|             2|
|2020-01-15 04:00:00| 179|             19.12|             3|
|2020-01-23 20:00:00| 260|            159.68|            12|
|2020-01-24 17:00:00|  92|177.72000000000003|            10|
|2020-01-12 14:00:00| 168|60.959999999999994|             4|
|2020-01-23 08:00:00|  5

In [6]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [7]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

C:\Program Files\spark-3.5.3-bin-hadoop3\python\pyspark\sql\dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [8]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [9]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [10]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [11]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [12]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [14]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  17|            195.03|                   9|220.20999999999998|                    8|
|2020-01-01 00:00:00|  65|199.48999999999998|                  10| 409.3500000000001|                   19|
|2020-01-01 00:00:00|  76|            143.78|                   4|             35.51|                    2|
|2020-01-01 00:00:00| 129| 273.2100000000001|                  19|            211.62|                   13|
|2020-01-01 00:00:00| 144|              NULL|                NULL| 2842.729999999999|                  158|
|2020-01-01 00:00:00| 157|             52.86|                   2|             57.06|                    2|
|2020-01-01 00:00:00| 158|  

In [13]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [15]:
df_join = spark.read.parquet('data/report/revenue/total')

In [16]:
df_join

DataFrame[hour: timestamp, zone: bigint, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [17]:
df_zones = spark.read.parquet('zones/')

In [18]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [19]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [21]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|  25|             531.0|                  26|324.34999999999997|                   16|        25| Brooklyn|         Boerum Hill|   Boro Zone|
|2020-01-01 00:00:00|  63|              51.9|                   2|              70.8|                    1|        63| Brooklyn|       Cypress Hills|   Boro Zone|
|2020-01-01 00:00:00|  69|              11.8|                   1|106.52000000000001|                    4|        69|    Bronx|East Concourse/Co...|   Boro Zone|
|2020-01-01 00:00:00| 

In [20]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')